In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import math

In [68]:
# round 1
mw1 = pd.read_stata("../../Data/lsms/Malawi/short_panel/HouseholdGeovariables_IHS3_Rerelease_10.dta", 
                   columns = ['ea_id','case_id','lat_modified','lon_modified'])
#mw1 = mw1.rename(columns = {"lat_modified":"lat", "lon_modified":"lon"})
mw1['round'] = 1

# get weights, dates and interview outcome 
mw1_a = pd.read_stata('../../Data/lsms/Malawi/short_panel/HH_MOD_A_FILT_10.dta', convert_categoricals = False)
mw1_a = mw1_a.rename(columns = {"hh_a23_1":"date_visit_1", "hh_a23_2":"date_visit_2"})
mw1_a = mw1_a[['case_id','ea_id','hh_wgt','reside','date_visit_1']]

# get the asset data and consumption data
mw_cons = pd.read_csv("../../Data/lsms/Malawi/wave_1/Panel/Round 1 (2010) Consumption Aggregate.csv", 
                      usecols = ['case_id','hhsize','adulteq','rexpagg','pcrexpagg',
                                 'absolute_povline','extreme_povline','poor','epoor'])
mw_house = pd.read_csv("../../Data/lsms/Malawi/wave_1/Panel/Household/hh_mod_f.csv")
mw_house = mw_house.rename(columns = {'hh_f07':'wall_material',
                                      'hh_f08':'roof_material',
                                      'hh_f09':'floor_material',
                                      'hh_f10':'n_rooms',
                                      'hh_f11':'lighting_fuel',
                                      'hh_f12':'cooking_fuel',
                                      'hh_f19':'electricity',
                                      'hh_f34':'n_cellphones',
                                      'hh_f36':'water_source',
                                      'hh_f41':'toilet_facility'})
mw_house = mw_house[['case_id','wall_material','roof_material','floor_material','n_rooms','lighting_fuel',
                    'cooking_fuel','electricity','n_cellphones','water_source','toilet_facility']]
mw_ass = pd.read_csv("../../Data/lsms/Malawi/wave_1/Panel/Household/hh_mod_l.csv")

# for assets keep the Radio (507), TV (509),Bike (516), Motorbike (517), Car (518)
mask = [i in [507,509,516,517,518] for i in mw_ass['hh_l02']]
mw_ass = mw_ass.loc[mask,:].reset_index(drop = True)
mw_ass['own'] = [1 if i == 1 else 0 for i in mw_ass['hh_l01']]
code_asset_dict = {507:'radio',509:'tv',516:'bike',517:'motorbike',518:'car'}
mw_ass['asset'] = [code_asset_dict[i] for i in mw_ass['hh_l02']]
mw_ass = mw_ass[['case_id','asset','own']]
mw_ass = mw_ass.pivot(index = 'case_id',columns = 'asset', values = 'own').reset_index()
mw_ass.columns.name = None